# Semantic Kernel AI智能体入门教程

只需几步，即可学会使用Semantic Kernel构建AI智能体。本教程涵盖了创建智能体、为其添加工具以及管理对话等核心要素。

在Semantic Kernel中，智能体有三个基本组成部分：
1. **智能体类 (Agent Class)**: 所有智能体类型都继承自该基类。常见的智能体类型包括：
   - `ChatCompletionAgent` (使用标准的聊天补全API)
   - `OpenAIAssistantAgent` (利用OpenAI的助手API及其内置工具)
   - `AzureAIAgent` (为企业场景与Azure AI服务集成)
   - `CopilotStudioAgent` (连接到Microsoft Copilot Studio的工作流)
2. **智能体线程 (Agent Thread)**: 负责维护对话历史和状态。这至关重要，因为智能体需要根据之前的消息上下文来做出明智的决策。主要有两种方法：
   - **服务端管理状态**: 由Azure AI等智能体服务在服务器端存储对话历史，通过线程ID进行访问。
   - **应用端管理状态**: 由你的应用程序自己维护完整的聊天记录，并在每次调用时传递给智能体。
3. **智能体编排 (Agent Orchestration)**: 框架提供了预置的模式，用于协调多个智能体以处理单个智能体无法有效管理的复杂工作流。
   - **顺序型 (Sequential)**: 智能体按顺序依次执行，如同一个文档处理流水线。
   - **并发型 (Concurrent)**: 多个智能体同时工作，例如处理客户咨询时，账单智能体和账户智能体并行工作。
   - **交接型 (Handoff)**: 智能体根据专业领域将控制权传递给彼此，如客服分流给专家智能体。
   - **群聊型 (Group Chat)**: 智能体在共享的对话中协作，如领域专家共同进行项目规划。

智能体利用Semantic Kernel的插件系统来访问工具、数据库等。智能体也可以使用YAML文件进行定义。

## 设置

安装所需软件包并配置你的环境：

In [ ]:
# 安装软件包
!pip install semantic-kernel python-dotenv

# 导入我们需要的一切
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

print("✅ 设置完成！")

In [ ]:
import os
from dotenv import load_dotenv

# 从.env文件加载环境变量
load_dotenv()

# 检查API密钥是否已配置
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("⚠️  请将OPENAI_API_KEY添加到您的.env文件中！")
    print("   创建一个.env文件，内容为: OPENAI_API_KEY=your_actual_key_here")
else:
    print("✅ API密钥已配置")

## 步骤 1: 创建一个简单的智能体

让我们从基础开始——一个会聊天的智能体。我们只需让内核（我们的协调器）访问聊天服务，该服务利用OpenAI的聊天补全API端点。稍后我们将添加更多工具。

In [ ]:
# 创建服务和内核
chat_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    api_key=OPENAI_API_KEY
)

kernel = Kernel()
kernel.add_service(chat_service)

# 创建一个基础智能体
agent = ChatCompletionAgent(
    kernel=kernel,
    name="助手",
    instructions="你是一个乐于助人且友好的助手。"
)

# 测试它
response = await agent.get_response("你好！我能帮你什么吗？")
print(f"🤖 {agent.name}: {response.content}")

## 步骤 2: 添加工具 (函数)

现在，让我们为我们的智能体赋予一些实用的能力。我们将给它一个天气查询函数和一个计算器。这两个都是我们在代码中定义的函数，并作为工具提供给内核。

In [ ]:
# 定义有用的函数
@kernel_function(description="获取一个城市的当前天气")
def get_weather(city: str) -> str:
    """模拟的天气函数 - 将来可以用真实的API调用替换。"""
    weather_data = {
        "伦敦": "多云, 15°C",
        "巴黎": "晴, 22°C", 
        "东京": "雨, 18°C",
        "纽约": "部分多云, 20°C"
    }
    return weather_data.get(city.lower(), f"无法获取 {city} 的天气数据")

@kernel_function(description="计算简单的数学表达式")
def calculate(expression: str) -> str:
    """用于基础数学计算的安全计算器。"""
    try:
        # 为安全起见，只允许基础的数学运算
        allowed_chars = "0123456789+-*/(). "
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return f"{expression} = {result}"
        else:
            return "抱歉，我只能进行基础的数学运算 (+, -, *, /, 括号)"
    except:
        return "抱歉，我无法计算。请检查您的表达式。"

# 将函数添加到内核
kernel.add_function(plugin_name="tools", function=get_weather)
kernel.add_function(plugin_name="tools", function=calculate)

# 创建增强型智能体
enhanced_agent = ChatCompletionAgent(
    kernel=kernel,
    name="智能助手",
    instructions="""
    你是一个拥有天气查询和计算器功能的乐于助人的助手。
    
    - 当被问及特定城市的天气时，使用 get_weather 函数。
    - 对于数学问题，使用 calculate 函数。
    - 保持友好，并解释你正在做什么。
    """
)

print("✅ 带有工具的增强型智能体已创建！")

## 步骤 3: 测试带工具的智能体

让我们看看我们的智能体如何使用它的工具：

In [ ]:
# 测试天气函数
print("🌤️ 测试天气函数:")
response = await enhanced_agent.get_response("伦敦的天气怎么样？")
print(f"🤖 {enhanced_agent.name}: {response.content}\n")

# 测试计算器函数
print("🧮 测试计算器函数:")
response = await enhanced_agent.get_response("25 * 4 + 10 是多少？")
print(f"🤖 {enhanced_agent.name}: {response.content}\n")

# 测试一般对话
print("💬 测试一般对话:")
response = await enhanced_agent.get_response("给我讲一个关于AI的有趣事实")
print(f"🤖 {enhanced_agent.name}: {response.content}")

## 步骤 4: 带记忆的对话

对于需要记住之前消息的对话，我们将使用Semantic Kernel内置的内存管理，但这将取决于上下文窗口的大小。

内存问题可以通过总结过去的对话或使用像RAG这样的长期记忆来解决。还有一种学习记忆，我们希望智能体能从所有过去的交互中学习，这也是通过RAG实现的，我们将成功的解决方案示例存储起来，并在类似情况下检索它们。

In [ ]:
async def chat_with_memory():
    """演示带记忆的对话。"""
    
    print("💭 带记忆的对话演示")
    print("=" * 40)
    
    # 相互关联的消息
    messages = [
        "嗨！我正计划去巴黎旅行。",
        "那里的天气怎么样？",
        "听起来不错！你能为我的预算计算一下 150 * 7 吗？",
        "太好了，这应该够我待一周了。谢谢！"
    ]
    
    thread = None  # 这将存储对话历史
    
    for msg in messages:
        print(f"👤 用户: {msg}")
        
        # 智能体通过线程记住之前的消息
        response = await enhanced_agent.get_response(messages=msg, thread=thread)
        print(f"🤖 {enhanced_agent.name}: {response.content}\n")
        
        # 更新线程以保持对话历史
        thread = response.thread

# 运行对话演示
await chat_with_memory()

## 步骤 5: 查看工具的实际使用情况 (高级)

观察当你的智能体使用工具时究竟发生了什么：

In [ ]:
async def show_tool_usage():
    """显示详细的工具执行过程。"""
    
    print("🔧 工具使用演示")
    print("=" * 25)
    
    # 回调函数以查看工具调用
    async def log_tool_calls(message):
        from semantic_kernel.contents import FunctionCallContent, FunctionResultContent
        
        for item in message.items or []:
            if isinstance(item, FunctionCallContent):
                print(f"  🔧 正在调用: {item.name}({item.arguments})")
            elif isinstance(item, FunctionResultContent):
                print(f"  ✅ 结果: {item.result}")
    
    user_input = "东京的天气怎么样？15 + 27 是多少？"
    print(f"👤 用户: {user_input}\n")
    
    # 使用 invoke 查看中间步骤
    async for response in enhanced_agent.invoke(
        messages=user_input,
        on_intermediate_message=log_tool_calls
    ):
        print(f"\n🤖 最终响应: {response.content}")

# 运行工具使用演示
await show_tool_usage()

## 步骤 6: 流式响应

对于实时响应（像ChatGPT那样）：

In [ ]:
async def streaming_demo():
    """显示流式响应。"""
    
    print("\n🌊 流式响应演示")
    print("=" * 20)
    
    print("👤 用户: 写一首关于编码的短诗")
    print("🤖 助手: ", end="", flush=True)
    
    # 逐字流式输出响应
    async for chunk in enhanced_agent.invoke_stream(
        messages="写一首关于编码的短诗"
    ):
        print(chunk.content, end="", flush=True)
    
    print("\n")  # 完成后换行

# 运行流式响应演示
await streaming_demo()


## 总结: 你学到了什么

In [ ]:
print("🎓 你构建了什么:")
print("=" * 30)

summary = [
    "✅ 使用OpenAI的基础AI智能体",
    "✅ 用于天气和数学的自定义工具/函数", 
    "✅ 对话记忆管理",
    "✅ 工具执行监控",
    "✅ 实时流式响应"
]

for item in summary:
    print(item)

print("\n🚀 核心概念:")
concepts = {
    "智能体 (Agent)": "能够推理、记忆和使用工具的AI",
    "内核 (Kernel)": "管理AI服务和函数",
    "函数 (Functions)": "扩展智能体能力的工具", 
    "线程 (Thread)": "维护对话历史",
    "流式 (Streaming)": "实时响应生成"
}

for concept, description in concepts.items():
    print(f"• {concept}: {description}")

print("\n💡 下一步:")
print("• 尝试不同的模型 (gpt-4, gpt-3.5-turbo)")
print("• 为你的用例创建自定义函数")
print("• 探索多智能体对话")
print("• 为生产安全添加护栏")

## 步骤 7: 顺序智能体编排

现在让我们看看多个智能体如何在一个流水线中协同工作——每个智能体处理前一个智能体的输出。注意Semantic Kernel让这个过程变得多么简单：

In [ ]:
# 导入编排组件
from semantic_kernel.agents import SequentialOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.contents import ChatMessageContent

print("🔗 正在设置顺序智能体流水线")
print("=" * 45)

In [ ]:
# 为营销流水线创建专门的智能体
def create_marketing_pipeline():
    """创建三个按顺序协同工作的智能体。"""
    
    # 智能体 1: 提取关键信息
    concept_extractor = ChatCompletionAgent(
        name="概念提取器",
        instructions="""
        你是一位市场分析师。根据产品描述，识别：
        - 关键特性 (要点形式)
        - 目标受众
        - 独特卖点
        
        用清晰的标题格式化你的输出。
        """,
        kernel=kernel
    )
    
    # 智能体 2: 撰写营销文案
    copywriter = ChatCompletionAgent(
        name="文案撰写员", 
        instructions="""
        你是一位营销文案撰写员。根据提供的分析，撰写引人注目的营销文案（约100-150字）。使其引人入胜，并突出关键优势。只输出营销文案。
        """,
        kernel=kernel
    )
    
    # 智能体 3: 润色和格式化
    editor = ChatCompletionAgent(
        name="编辑",
        instructions="""
        你是一位编辑。对营销文案进行润色：
        - 修正语法和清晰度
        - 确保语调一致
        - 使其更具吸引力
        - 输出最终的润色版本
        """,
        kernel=kernel
    )
    
    return [concept_extractor, copywriter, editor]

# 创建流水线
marketing_agents = create_marketing_pipeline()
print(f"✅ 已创建 {len(marketing_agents)} 个专门的智能体:")
for agent in marketing_agents:
    print(f"   • {agent.name}")

In [ ]:
# 设置回调函数以查看每个智能体的工作
def agent_callback(message: ChatMessageContent) -> None:
    """显示每个智能体的产出。"""
    print(f"\n🤖 {message.name}:")
    print("-" * 30)
    print(message.content)
    print()

# 创建顺序编排
sequential_pipeline = SequentialOrchestration(
    members=marketing_agents,
    agent_response_callback=agent_callback
)

print("🔗 顺序流水线已创建！")

In [ ]:
# 运行顺序流水线
async def run_marketing_pipeline():
    """执行顺序智能体流水线。"""
    
    print("🚀 正在运行营销流水线")
    print("=" * 35)
    
    # 启动运行时
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        # 输入: 产品描述
        product_description = (
            "一款带温度显示、应用连接、补水提醒和防漏设计的智能水瓶。由不含BPA的材料制成。"
        )
        
        print(f"📝 输入产品: {product_description}\n")
        print("正在通过流水线处理...")
        
        # 运行顺序编排
        result = await sequential_pipeline.invoke(
            task=product_description,
            runtime=runtime
        )
        
        # 获取最终结果
        final_output = await result.get(timeout=30)
        
        print("🎯 最终营销文案:")
        print("=" * 40)
        print(final_output)
        
    finally:
        # 清理
        await runtime.stop_when_idle()

# 执行流水线
await run_marketing_pipeline()

In [ ]:
# 使用不同产品的快速演示
async def quick_pipeline_demo():
    """使用另一个产品进行快速演示。"""
    
    print("\n🔄 快速流水线演示 #2")
    print("=" * 30)
    
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        # 不同的产品
        product = "具有30小时电池续航、降噪功能和防汗设计的无线耳塞"
        
        print(f"📝 产品: {product}")
        
        # 不带详细日志的简单流水线
        simple_pipeline = SequentialOrchestration(members=marketing_agents)
        
        result = await simple_pipeline.invoke(
            task=product,
            runtime=runtime
        )
        
        final_copy = await result.get(timeout=30)
        print(f"\n🎯 最终文案:\n{final_copy}")
        
    finally:
        await runtime.stop_when_idle()

# 运行快速演示
await quick_pipeline_demo()


## 总结: 你学到了什么

In [ ]:
print("🎓 完整教程总结:")
print("=" * 35)

summary = [
    "✅ 使用OpenAI的基础AI智能体",
    "✅ 用于天气和数学的自定义工具/函数", 
    "✅ 对话记忆管理",
    "✅ 工具执行监控",
    "✅ 实时流式响应",
    "✅ 顺序智能体编排"
]

for item in summary:
    print(item)

print("\n🚀 核心概念:")
concepts = {
    "智能体 (Agent)": "能够推理、记忆和使用工具的AI",
    "内核 (Kernel)": "管理AI服务和函数",
    "函数 (Functions)": "扩展智能体能力的工具", 
    "线程 (Thread)": "维护对话历史",
    "流式 (Streaming)": "实时响应生成",
    "顺序编排 (Sequential Orchestration)": "智能体按顺序处理输出的流水线"
}

for concept, description in concepts.items():
    print(f"• {concept}: {description}")

print("\n💡 智能体模式:")
patterns = [
    "单智能体: 一个智能体处理整个工作流",
    "顺序型: 智能体在流水线中工作 (A → B → C)",
    "并发型: 多个智能体同时工作", 
    "管理者型: 中央智能体协调专家",
    "交接型: 智能体之间传递控制权"
]

for pattern in patterns:
    print(f"• {pattern}")

print("\n🎯 何时使用顺序编排:")
use_cases = [
    "• 文档处理 (提取 → 总结 → 格式化)",
    "• 内容创作 (研究 → 撰写 → 编辑)", 
    "• 数据分析 (收集 → 分析 → 可视化)",
    "• 代码审查 (分析 → 建议 → 验证)"
]

for use_case in use_cases:
    print(use_case)

---

**🔑 关键要点:**
- **单智能体**: 非常适合简单的任务和学习。
- **顺序编排**: 非常适合每一步都建立在前一步基础上的多步骤工作流。
- **专业化**: 每个智能体都专注于自己最擅长的事情。
- **流水线优势**: 通过专业化处理提高质量。
- **实际应用**: 文档处理、内容创作、数据分析。

本教程涵盖了从基础智能体到复杂的多智能体流水线的所有内容！